In [11]:
# ! pip install  --index "http://pypi/simple" --trusted-host pypi  hyperopt

In [2]:
import sys, os, gc
import numpy as np
import pandas as pd
sys.path.append("./global")
from helper import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TRAIN_DTYPE, ORI_TEST_NAMES, ORI_TEST_DTYPE, AnalysisCSV, OFFLINE, cal_sim
import dist_utils, ngram_utils
test_file = "./stage1/input/test.csv"
base_feature_save_dir = "./stage1/output/"
if OFFLINE:
    base_prefix = "debug_"
    train_file = "./stage1/input/train_last_50w.csv"
    CHUNK_SIZE = 500000
else:
    base_prefix = "online_"
    train_file = "./stage1/input/train_last_1000w.csv"
    CHUNK_SIZE = 1000000
print(OFFLINE)

True


In [3]:
AnalysisCSV("./stage2/output/feature_selection/features_weight_simple.csv", print_rows=100)

--->file shape:
(48, 5)
--->100 rows of file:
                                           Unnamed: 0  Ridge  Lasso  \
0                    debug_trainTextMining_text_q-len  0.002  0.000   
1                    debug_trainTextMining_text_t-len  0.002  0.030   
2             debug_trainTextMining_text_qt-len-radio  0.099  0.000   
3             debug_trainTextMining_text_tq-len-radio  0.001  0.000   
4                debug_trainTextMining_unigrams_q-len  0.010  0.000   
5                debug_trainTextMining_unigrams_t-len  0.008  0.000   
6         debug_trainTextMining_unigrams_qt-len-radio  0.076  0.000   
7         debug_trainTextMining_unigrams_tq-len-radio  0.000  0.000   
8               debug_trainTextMining_text_dice_ratio  0.041  0.000   
9           debug_trainTextMining_text_edit_seq_ratio  0.166  0.000   
10          debug_trainTextMining_text_edit_set_ratio  0.075  0.000   
11                   debug_trainTextMining_text_ratio  0.001  0.000   
12           debug_trainTextMin

In [24]:
# AnalysisCSV("./stage1/input/train_last_50w.csv", names=ORI_TRAIN_NAMES)
# AnalysisCSV("./stage1/input/train_last_1000w.csv", names=ORI_TRAIN_NAMES)
# AnalysisCSV("./stage1/input/test.csv", names=ORI_TRAIN_NAMES)

In [30]:
# AnalysisCSV("./stage1/output/magic/online_trainMagic_ctrRate.csv.gz")

## 目录查看

In [12]:
!ls -alh stage1/input
!ls -alh stage1/output/text_mining_feature
!ls -alh stage1/output/vector_space
!ls -alh stage1/output/word2vec
# PrintCSV(train_file, names=ORI_TRAIN_NAMES)
# !cd stage1/input; mv word2vec_sentences.txt word2vec_sentences_1000w.txt
# !cd stage1/input; rm word2vec_sentences_1000w.txt
# ! rm stage1/output/vector_space/online_trainVectorSpace_lsi_feature.npz

total 2.5G
drwxr-xr-x 2 kesci root  4.0K Jun 18 10:46 .
drwxr-xr-x 6 kesci users 4.0K Jun  9 06:09 ..
-rw-r--r-- 1 kesci users 426M Jun 14 13:13 test.csv
-rw-r--r-- 1 kesci users 880M Jun 14 13:11 train_last_1000w.csv
-rw-r--r-- 1 kesci users  44M Jun 15 12:48 train_last_50w.csv
-rw-r--r-- 1 kesci users  44M Jun 15 06:06 word2vec.model
-rw-r--r-- 1 kesci users 546M Jun 15 06:06 word2vec.model.trainables.syn1neg.npy
-rw-r--r-- 1 kesci users 546M Jun 15 06:06 word2vec.model.wv.vectors.npy
total 6.1G
drwxr-xr-x 2 kesci root  4.0K Jun 17 14:42 .
drwxr-xr-x 6 kesci root  4.0K Jun 17 11:50 ..
-rw-r--r-- 1 kesci users  31M Jun 17 06:44 debug_train_text_mining_feature_len.csv
-rw-r--r-- 1 kesci users  31M Jun 17 06:46 debug_trainTextMining_feature_len.csv
-rw-r--r-- 1 kesci users  94M Jun 16 16:14 debug_train_text_mining_feature_ngramSimilar.csv
-rw-r--r-- 1 kesci users 109M Jun 17 06:59 debug_trainTextMining_feature_ngramSimilar.csv
-rw-r--r-- 1 kesci users 243M Jun 15 14:48 online_test_text_

In [3]:
!ls -alh stage2/input
!ls -alh stage2/output

total 2.1G
drwxr-xr-x 2 kesci root  4.0K Jun 17 15:38 .
drwxr-xr-x 6 kesci users 4.0K Jun  9 13:28 ..
-rw-r--r-- 1 kesci users 2.2K Jun 17 07:11 debug_feature_names.csv
-rw-r--r-- 1 kesci users 134M Jun 17 07:11 debug_train_concat_feature.npy
-rw-r--r-- 1 kesci users 7.7M Jun 16 06:08 debug_train_for_cal_auc.npy
-rw-r--r-- 1 kesci users 3.9M Jun 15 15:30 debug_train_labels.npy
-rw-r--r-- 1 kesci users 1.4K Jun 17 15:39 online_feature_names.csv
-rw-r--r-- 1 kesci users 611M Jun 17 15:39 online_test_concat_feature.npy
-rw-r--r-- 1 kesci users 1.2G Jun 17 15:38 online_train_concat_feature.npy
-rw-r--r-- 1 kesci users  77M Jun 15 15:13 online_train_labels.npy
total 52K
drwxr-xr-x 4 kesci root  4.0K Jun 17 16:54 .
drwxr-xr-x 6 kesci users 4.0K Jun  9 13:28 ..
-rw-r--r-- 1 kesci users 1.3K Jun 16 16:44 lgb_gbdt_debug_len_ngramSimilarFuzzy_lsiDis_lsiFeat_0.5593_1560703452.json
-rw-r--r-- 1 kesci users 1.4K Jun 17 07:31 lgb_gbdt_debug_len_ngramSimilarFuzzy_lsiDis_lsiFeat_0.5593_1560756714.json

In [21]:
!ls -alh stage2/output/train
!ls -alh stage2/output/test

total 851M
drwxr-xr-x 2 kesci root  4.0K Jun 18 15:29 .
drwxr-xr-x 4 kesci root  4.0K Jun 17 16:54 ..
-rw-r--r-- 1 kesci users 3.9M Jun 16 07:07 lgb_gbdt_1560668620_auc0.555900_loss0.563300_std0.000700.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 07:11 lgb_gbdt_1560668860_auc0.556500_loss0.563300_std0.000700.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 07:15 lgb_gbdt_1560669100_auc0.556200_loss0.563400_std0.000800.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 07:19 lgb_gbdt_1560669344_auc0.555800_loss0.563300_std0.000700.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 07:23 lgb_gbdt_1560669585_auc0.556100_loss0.563300_std0.000700.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 11:30 lgb_gbdt_1560684432_auc0.563697_loss0.562534_std0.001082.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 11:34 lgb_gbdt_1560684648_auc0.563070_loss0.562535_std0.000942.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 11:37 lgb_gbdt_1560684869_auc0.561621_loss0.562400_std0.000808.npy
-rw-r--r-- 1 kesci users 3.9M Jun 16 11:41 lgb_gbdt_1560685083_auc

In [41]:
!cat stage2/output/lgb_gbdt_debug_len_ngram_similarity_0.5633_1560669808.json

{
 "state": 2,
 "tid": 1,
 "spec": null,
 "result": {
  "loss": 0.5632760167257869,
  "auc": 0.5564878203367812,
  "loss_variance": 0.0007281675021687126,
  "status": "ok",
  "file_name": "lgb_gbdt_1560668860_auc0.556500_loss0.563300_std0.000700.npy",
  "q_auc": 0.5204636949875794
 },
 "misc": {
  "tid": 1,
  "cmd": [
   "domain_attachment",
   "FMinIter_Domain"
  ],
  "workdir": null,
  "idxs": {
   "bagging_fraction": [
    1
   ],
   "bagging_freq": [
    1
   ],
   "feature_fraction": [
    1
   ],
   "learning_rate": [
    1
   ],
   "max_bin": [
    1
   ],
   "min_data_in_leaf": [
    1
   ],
   "num_iterations": [
    1
   ],
   "num_leaves": [
    1
   ]
  },
  "vals": {
   "bagging_fraction": [
    0.9
   ],
   "bagging_freq": [
    2.0
   ],
   "feature_fraction": [
    0.9
   ],
   "learning_rate": [
    0.14
   ],
   "max_bin": [
    2
   ],
   "min_data_in_leaf": [
    20.0
   ],
   "num_iterations": [
    80.0
   ],
   "num_leaves": [
    10.0
   ]
  }
 },
 "exp_key": nu

## 结果提交

In [10]:
res_id = "lgb_gbdt_1561213593_auc0.672238_loss0.533564_std0.003697"
out_res_file = "./stage2/output/test/"+res_id+".npy"
test_file = "./stage1/input/test.csv"

In [11]:
out_test_res = np.load(out_res_file)
test_csv = ReadCSV(test_file, names=ORI_TEST_NAMES, dtype=None, iterator=False)
res = pd.DataFrame({
    "query_id": test_csv["query_id"],
    "query_title_id": test_csv["query_title_id"],
    "label":out_test_res
})
res.to_csv("./global/submits/"+res_id+".csv", index=None, header=None)

In [27]:
! ls -lsh ./global/submits

total 2.1G
7.5M -rwxr-xr-x 1 kesci users 7.5M May 25 15:15 kesci_submit
136M -rw-r--r-- 1 kesci users 136M Jun  8 12:26 lgb_gbdt_1559990841_0.547674_0.003729.csv
136M -rw-r--r-- 1 kesci users 136M Jun  9 00:27 lgb_gbdt_1560015399_0.548471_0.003714.csv
136M -rw-r--r-- 1 kesci users 136M Jun  9 14:20 lgb_gbdt_1560088510_0.559713_0.559681_0.559278_0.000611.csv
136M -rw-r--r-- 1 kesci users 136M Jun  9 23:59 lgb_gbdt_1560101261_0.558418_0.558846_0.561068_0.001116.csv
136M -rw-r--r-- 1 kesci users 136M Jun 10 07:23 lgb_gbdt_1560145986_0.559661_0.559640_0.559232_0.000524.csv
136M -rw-r--r-- 1 kesci users 136M Jun 12 13:11 lgb_gbdt_1560343909_0.566367_0.558761_0.001295.csv
136M -rw-r--r-- 1 kesci users 136M Jun 14 12:13 lgb_gbdt_1560513856_0.566747_0.561561_0.000980.csv
137M -rw-r--r-- 1 kesci users 137M Jun 16 03:13 lgb_gbdt_1560654538_0.553203_0.564397_0.000363.csv
136M -rw-r--r-- 1 kesci users 136M Jun 16 05:43 lgb_gbdt_1560662390_0.557880_0.563663_0.000493.csv
136M -rw-r--r-- 1 kesci user

In [12]:
!cd ./global/submits; ./kesci_submit -token 490475a1ae106f67 -file lgb_gbdt_1561213593_auc0.672238_loss0.533564_std0.003697.csv

Kesci Submit Tool
Result File: lgb_gbdt_1561213593_auc0.672238_loss0.533564_std0.003697.csv (135.86 MiB)
Uploaded.       
Submit Success.
{"Stage":0,"Status":0,"ShownInHistory":true,"IsAucResult":true,"Selected":false,"_id":"5d0e3e0f0a628a002bf7d04f","Competition":"5cc51043f71088002c5b8840","Team":"5cda7d99b4de58002b94c3f8","UploadDate":"2019-06-22T14:41:19.129Z","Final":true,"Response":"","SubmissionResults":[],"IP":"52.82.19.99","FingerPrint":"","UserAgent":"Go-http-client/1.1","ResultFileName":"15612144660591b06ab.csv","ResultFileRealName":"lgb_gbdt_1561213593_auc0.672238_loss0.533564_std0.003697.csv","ResultFileSize":0,"ReviewInfos":[],"__v":0}



## 进程管理

In [71]:
! ps aux
# ! kill -9 4430

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
kesci         1  0.0  0.0 374096 100648 ?       Ssl  Jun19   1:55 /opt/conda/bin
kesci         7  0.0  0.0 121344  5932 ?        Sl   Jun19   0:00 gotty --port 8
kesci       109  0.0  0.0      0     0 ?        Z    Jun19   0:00 [python] <defu
kesci       296  0.0  0.0      0     0 ?        Z    Jun19   0:00 [python] <defu
kesci       529  0.0  0.0      0     0 ?        Z    Jun19   0:00 [python] <defu
kesci       601  0.0  0.0      0     0 ?        Z    Jun19   0:00 [python] <defu
kesci       740  0.0  0.0      0     0 ?        Z    Jun19   0:00 [python] <defu
kesci      1043  0.0  0.0      0     0 ?        Z    Jun20   0:00 [python] <defu
kesci      1194  0.0  0.0      0     0 ?        Z    Jun20   0:00 [python] <defu
kesci      1344  0.0  0.0      0     0 ?        Z    Jun20   0:00 [python] <defu
kesci      1484  0.0  0.0      0     0 ?        Z    Jun20   0:00 [python] <defu
kesci      1624  0.0  0.0      0   

## 数据准备

1. 取后1000w条数据用以训练
2. 将test.csv移动到stage1/input文件夹

In [1]:
# !tail -n 500000 "./stage1/input/train_last_1000w.csv" > "./stage1/input/train_last_50w.csv"

In [9]:
# ! cp "/home/kesci/input/bytedance/first-round/test.csv" "./stage1/input/test.csv"